<a href="https://colab.research.google.com/github/kodai-utsunomiya/memorization-and-generalization/blob/main/numerical_experiments/1_time_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $(k, d)$-Sparse Parity Task

$d$-個の数字の内の $k$ 個の数字のパリティを計算する（$k \le d$）という問題

# データセット

---

- $\mathcal{D}_{k, d} = \{ (\boldsymbol{x}_i , y_i) \}_{i=1}^n$
    - $n$ 個の学習データ
    - $\boldsymbol{x}_i \in \{ 0,1 \}^d$：バイナリーベクトル．$\boldsymbol{x}_i \sim \text{Unif} \left( \{0,1\}^d \right)$
    - $y_i = \left(\sum_{i}^k x^{(i)} \right) \text{mod} \hspace{2mm} 2$ ：最初の $k$  個の数字（clean digits）のパリティ

<br>

  - $\boldsymbol{x}_i$ の残りの $d-k$ 個の数字（noisy digits）は $y_i$ とは無関係

<br>

- 例：
    - $(3, 30)$-sparse parity dataset
        - $\boldsymbol{x}_1$：<font color="blue">000</font>$110010110001010111001001011$，  $y_1 = 0$
        - $\boldsymbol{x}_2$：<font color="blue">010</font>$110010110001010111001001011$，  $y_2 = 1$
            
            $\hspace{2mm} \vdots$
            

<br>

- 学習データをまとめて，$\mathcal{X} = \left[ \boldsymbol{x}_1, \ldots, \boldsymbol{x}_n \right] \in \mathbb{R}^{n \times d}$，$\mathcal{Y} = \left[ y_1 \ldots, y_n \right] \in \mathbb{R}^n$ と行列表記

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset

class BinaryDataset(Dataset):
    def __init__(self, n, k, train_size, test_size, seed):
        """
        データセットの初期化

        Parameters:
        - n: バイナリ文字列の長さ
        - k: 出力ラベルの計算に使用する最初の k 個のビット
        - train_size: 訓練データのサイズ
        - test_size: テストデータのサイズ
        - seed: ランダムシード
        """
        np.random.seed(seed)
        self.n = n
        self.k = k
        self.train_size = train_size
        self.test_size = test_size
        self.total_size = train_size + test_size

        # ユニークなバイナリ文字列を生成
        self.unique_binary_strings = self._generate_unique_binary_strings()

        # 入力データと出力ラベルを準備
        self.inputs, self.outputs = self._prepare_data()

        # データのインデックスをシャッフル
        self.indices = np.random.permutation(len(self.inputs))

        # 訓練データとテストデータのインデックスを分割
        self.train_indices = self.indices[:self.train_size]
        self.test_indices = self.indices[self.train_size:]

    def _generate_unique_binary_strings(self):
        """
        ユニークなバイナリ文字列を生成するヘルパーメソッド

        Returns:
        - list: ユニークなバイナリ文字列のリスト．各バイナリ文字列は長さ n のタプル
        """
        # ユニークなバイナリ文字列を保存するための空のセットを作成
        unique_binary_strings = set()

        # 必要な数のユニークなバイナリ文字列が得られるまで繰り返す
        while len(unique_binary_strings) < self.total_size:
            # 長さ n のバイナリ文字列を生成
            # np.random.randint(2, size=self.n) は 0 または 1 の整数を含む長さ n の配列を生成
            binary_string = tuple(np.random.randint(2, size=self.n))
            unique_binary_strings.add(binary_string)
        return list(unique_binary_strings)

    def _prepare_data(self):
        """
        入力データと出力ラベルを準備するヘルパーメソッド

        Returns:
        - tuple: (入力データ, 出力ラベル)
          - 入力データ: バイナリ文字列を NumPy 配列として保持し，最後にバイアス列を追加
          - 出力ラベル: 最初の k ビットの合計の 2 で割った余りとして計算
        """
        # バイナリ文字列をNumPy配列に変換
        inputs = np.array(self.unique_binary_strings, dtype=np.float32)

        ########## この有無でどのくらい影響がある ???
        # # 各サンプルをノルムで割って球状に正規化
        # norms = np.linalg.norm(inputs, axis=1, keepdims=True)
        # inputs = inputs / norms

        # 出力ラベルを計算 (最初の k ビットの合計を 2 で割った余り)
        outputs = np.sum(inputs[:, :self.k], axis=-1) % 2

        ## 入力データにバイアス用の列を追加
        # ones_column = np.ones((inputs.shape[0], 1), dtype=np.float32)
        #inputs = np.concatenate((inputs, ones_column), axis=1)
        return inputs, outputs

    def __len__(self):
        """
        データセットのサイズを返す

        Returns:
        - int: データセットの総サンプル数（訓練データとテストデータの合計）を返す
        """
        return self.total_size

    def __getitem__(self, idx):
        """
        インデックスに対応するデータを返す

        Parameters:
        - idx: 取得したいデータのインデックス

        Returns:
        - tuple: (入力データ, 出力ラベル)
          - 入力データ: PyTorchテンソルとして返す
          - 出力ラベル: PyTorchテンソルとして返す
        """
        input_data = torch.tensor(self.inputs[self.indices[idx]], dtype=torch.float32)
        output_data = torch.tensor(self.outputs[self.indices[idx]], dtype=torch.float32)
        return input_data, output_data

    def get_train_data(self):
        """
        訓練データのサブセットを返す

        Returns:
        - Subset: 訓練データのサブセット．このサブセットには訓練データのインデックスが含まれている
        """
        return torch.utils.data.Subset(self, self.train_indices)

    def get_test_data(self):
        """
        テストデータのサブセットを返す

        Returns:
        - Subset: テストデータのサブセット．このサブセットにはテストデータのインデックスが含まれている
        """
        return torch.utils.data.Subset(self, self.test_indices)

# モデル

---

$ F(\boldsymbol{w}, \boldsymbol{x}) \equiv \alpha \left\lbrack f(\boldsymbol{w}, \boldsymbol{x}) - f(\boldsymbol{w}_0, \boldsymbol{x}) \right\rbrack $ という形をしたモデルの学習を考える（これを予測器として使用し，$\boldsymbol{w}$を学習）

# FCクラス．<font color="green"> $f(\boldsymbol{x})$ の定義 </font>

### ネットワークの構造

1. **入力層**: 次元数 $d$ の入力を受け取る．
2. **隠れ層**: 層数 $L$ の隠れ層があり，各隠れ層のユニット数は $h$．
3. **出力層**: 最終層は出力がスカラー値である 1 次元のベクトルを生成．

<br>

### 層ごとの計算

1. **初期化**:
   - 隠れ層 $i$ の重み行列 $W_i$ は，次のように初期化：
     
     $
     W_i \sim \mathcal{N}(0, 1)
     $

     ここで，$W_i$ のサイズは $ h \times \text{hh}_{i}$ ．
     
     $\text{hh}_{i} $ は前の層の出力ユニット数．

   - メモリ効率を考慮し，重み行列を分割：
     
     \begin{aligned} W_i =  \begin{bmatrix}
        W_i^{(0)} \\
        W_i^{(1)} \\
        \vdots \\
        W_i^{(n-1)}
        \end{bmatrix}  \end{aligned}
     
     各部分行列 $W_i^{(j)}$ はサイズ $m \times \text{hh}_{i}$．ここで，$m$ は分割サイズ．

<br>

2. **順伝播計算**:
   - 入力テンソル $x$ は，初期の隠れ層で次のように変換：
     
     $
     x^{(0)} = x W_0^T / \sqrt{d}
     $

     ここで，$W_0$ は最初の隠れ層の重み行列．バイアス項がある場合，次のように加算：
     
     $
     x^{(0)} = x^{(0)} + b_0
     $

     その後，活性化関数 $ \sigma $ を適用：
     
     $
     x^{(1)} = \sigma(x^{(0)})
     $

   - 次の隠れ層も同様に計算．一般的に，隠れ層 $i$ の計算は次のようになる：
     
     $
     x^{(i)} = x^{(i-1)} W_i^T / \sqrt{h}
     $

     ここで，$W_i$ は現在の層の重み行列．バイアス項がある場合，次のように加算：

     $
     x^{(i)} = x^{(i)} + b_i
     $

     そして，活性化関数を適用：
     
     $
     x^{(i+1)} = \sigma(x^{(i)})
     $

   - 最終層では，次のように計算：
     
     $
     x^{(L)} = x^{(L-1)} W_L^T / h + b_L
     $
     
     ここで，$W_L$ は最終層の重み行列．出力テンソル $x$ を 1 次元に変換して返す：

     $
     x^{(L)} = x^{(L)} \text{view}(-1)
     $

In [2]:
import functools
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

"""
全結合ネットワーク（Fully Connected Network, FC）のクラスを定義．
任意の層数 L を持ち，各層のユニット数は h で指定．
活性化関数 act は任意に指定可能で，バイアス項の有無も指定可能．
"""

class FC(nn.Module):
    def __init__(self, d, h, L, act, bias=False):
        super().__init__()

        # ネットワークの初期化
        hh = d  # 入力の次元数
        for i in range(L):
            # 隠れ層の重み行列を正規分布で初期化
            W = torch.randn(h, hh)

            # メモリ効率を考慮し，重み行列を部分行列に分割して ParameterList に格納
            # next two line are here to avoid memory issue when computing the kerne
            n = max(1, 128 * 256 // hh)  # 分割サイズを計算
            W = nn.ParameterList([nn.Parameter(W[j: j+n]) for j in range(0, len(W), n)])

            # 分割した重み行列をレイヤーとして登録
            setattr(self, "W{}".format(i), W)

            # バイアス項が指定されている場合は，それをゼロで初期化して登録
            if bias:
                self.register_parameter("B{}".format(i), nn.Parameter(torch.zeros(h)))

            # 次のレイヤーの入力次元は現在の隠れ層のユニット数になる
            hh = h

        # 最終層の重み行列を初期化（出力がスカラー値なので次元は (1, h)）
        self.register_parameter("W{}".format(L), nn.Parameter(torch.randn(1, hh)))

        # バイアス項が指定されている場合は，最終層のバイアスをゼロで初期化
        if bias:
            self.register_parameter("B{}".format(L), nn.Parameter(torch.zeros(1)))

        # クラス変数としてレイヤー数，活性化関数，バイアスの有無を保持
        self.L = L
        self.act = act
        self.bias = bias

    def forward(self, x):
        # 順伝播計算
        for i in range(self.L + 1):
            # i 番目の層の重み行列を取得
            W = getattr(self, "W{}".format(i))

            # ParameterList 形式の重み行列をフルの行列に結合
            if isinstance(W, nn.ParameterList):
                W = torch.cat(list(W))

            # バイアス項が指定されている場合は，バイアスを取得
            if self.bias:
                B = self.bias * getattr(self, "B{}".format(i))
            else:
                B = 0

            # 現在の入力の次元数を取得
            h = x.size(1)

            if i < self.L:
                # 隠れ層での線形変換とスケーリング，そして活性化関数の適用
                x = x @ (W.t() / h ** 0.5)  # 重み行列との積（次元スケーリング）
                x = self.act(x + B)  # バイアス項を加えた後，活性化関数を適用
            else:
                # 最終層での線形変換（出力はスカラー値）
                x = x @ (W.t() / h) + B  # スカラー出力

        # 出力を 1 次元のテンソルに変換して返す
        return x.view(-1)

# ダイナミクス

In [28]:
"""
Dynamics that compares the angle of the gradient between steps and keep it small

- マージンに達したときに停止

2つの実装：
1. `train_regular` - 任意のモデルに対応
2. `train_kernel` - 線形モデル専用
"""

import copy
import itertools
import math
from time import perf_counter
import torch


def gradient(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False):
    '''
    `outputs` に対する `inputs` の勾配を計算する関数
    使用例:
    ```
    gradient(x.sum(), x)          # x の合計に対する勾配
    gradient((x * y).sum(), [x, y])  # x と y の要素ごとの積の合計に対する勾配
    ```

    :param outputs: 勾配を計算する対象の出力テンソル
    :param inputs: 勾配を計算したい入力テンソルのリストまたは単一テンソル
    :param grad_outputs: 出力テンソルの勾配を指定するためのオプション（通常は None で良い）
    :param retain_graph: 計算グラフを保持するかどうかを指定するフラグ（デフォルトは None）
    :param create_graph: 勾配の計算グラフを作成するかどうかを指定するフラグ（デフォルトは False）
    :return: 入力テンソルに対する勾配をフラットなテンソルとして返す
    '''

    # `inputs` がテンソルの場合はリストに変換
    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)

    # `torch.autograd.grad` 関数を使用して勾配を計算
    grads = torch.autograd.grad(outputs, inputs, grad_outputs,
                                allow_unused=True, # 計算に使用されないテンソルには勾配が計算されない
                                retain_graph=retain_graph, # 計算グラフを保持するかどうか
                                create_graph=create_graph) # 勾配の計算グラフを作成するかどうか

    # 勾配が None の場合は，同じサイズのゼロテンソルを代わりに使用
    grads = [x if x is not None else torch.zeros_like(y) for x, y in zip(grads, inputs)]

    # 勾配テンソルをフラットな形状に変換して連結
    return torch.cat([x.contiguous().view(-1) for x in grads])


def loglinspace(rate, step, end=None):
    """
    対数線形間隔での数値を生成するジェネレーター関数
    対数的に変化する間隔で数値を生成

    `rate` と `step` のパラメータを使って，新しい値を計算
    `end` が指定されていない場合は無限に数値を生成

    Arguments:
        rate (float): 対数的な変化の速度を制御するパラメータ
        step (float): 各ステップでの間隔の大きさ
        end (float, optional): 生成を停止する条件となる最大値．指定されない場合は無限に生成

    Yields:
        float: 現在の時間 `t` の値を生成
    """
    t = 0
    while end is None or t <= end:
        yield t  # 現在の時間 `t` の値を生成
        # 次の `t` を計算．ここで，`math.exp(-t * rate / step)` は指数関数的な減衰を表す
        t = int(t + 1 + step * (1 - math.exp(-t * rate / step)))


class ContinuousMomentum(torch.optim.Optimizer):
    """連続的なモーメンタムを実装

    - d/dt velocity = -1/tau (velocity + grad)
    - または
    - d/dt velocity = -mu/t (velocity + grad)

    - d/dt parameters = velocity
    """

    def __init__(self, params, dt, tau):
        """
        初期化メソッド

        Arguments:
            params (iterable): 最適化するパラメータのリスト
            dt (float): 時間ステップのサイズ
            tau (float): モーメンタムのタイムコンスタント
        """
        defaults = dict(dt=dt, tau=tau)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """単一の最適化ステップを実行

        Arguments:
            closure (callable, optional): モデルを再評価し，損失を返すクロージャ．多くの最適化器にはオプショナル．

        Returns:
            loss (Tensor or None): 損失の値．クロージャが指定された場合はその損失値を返す．
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            tau = group['tau']
            dt = group['dt']

            for p in group['params']:
                # 勾配がないパラメータはスキップ
                if p.grad is None:
                    continue

                param_state = self.state[p]
                # パラメータの状態が初めてのときは，時間 t を 0 に設定
                if 't' not in param_state:
                    t = param_state['t'] = 0
                else:
                    t = param_state['t']

                # モーメンタムの状態（速度）を初期化
                if tau != 0:
                    if 'velocity' not in param_state:
                        v = param_state['velocity'] = torch.zeros_like(p.data)
                    else:
                        v = param_state['velocity']

                # モーメンタムの計算
                if tau > 0:
                    # tau > 0 の場合の連続モーメンタムの計算
                    x = math.exp(-dt / tau)  # 時間の経過とともに減衰する係数
                    v.mul_(x).add_(-(1 - x), p.grad.data) # 速度に勾配を加える
                elif tau < 0:
                    # tau < 0 の場合の連続モーメンタムの計算
                    mu = -tau
                    x = (t / (t + dt)) ** mu  # 時間の経過に伴う係数
                    v.mul_(x).add_(-(1 - x), p.grad.data)  # 速度に勾配を加える
                else:
                    # tau = 0 の場合のシンプルな勾配降下
                    v = -p.grad.data

                # パラメータの更新
                p.data.add_(dt, v)    # パラメータに速度を加える
                param_state['t'] += dt    # 時間を進める

        return loss


def make_step(f, optimizer, dt, grad):
    """
    指定された勾配 `grad` を使用して，最適化ステップを実行

    Arguments:
        f (torch.nn.Module): トレーニングするモデル
        optimizer (torch.optim.Optimizer): 使用する最適化器
        dt (float): 時間刻み
        grad (torch.Tensor): 勾配テンソル
    """
    i = 0
    # モデルの全パラメータに対してループ
    for p in f.parameters():
        # パラメータの総要素数を取得
        n = p.numel()
        # 勾配テンソルを対応するパラメータに合わせてリシェイプし，割り当て
        p.grad = grad[i: i + n].view_as(p)
        i += n  # インデックスを次のパラメータに進める

    # 各パラメータグループに対して時間刻み `dt` を設定
    for param_group in optimizer.param_groups:
        param_group['dt'] = dt

    # 最適化ステップを実行
    optimizer.step()

    # 勾配をリセット（計算グラフから切り離し）
    for p in f.parameters():
        p.grad = None


def train_regular(f0, x, y, tau, max_walltime, alpha, loss, subf0, max_dgrad=math.inf, max_dout=math.inf):
    """
    一般的なモデルのトレーニング関数

    Arguments:
        f0 (torch.nn.Module): 初期モデル
        x (torch.Tensor): 入力データ
        y (torch.Tensor): 出力ラベル
        tau (float): モーメンタムパラメータ
        max_walltime (float): 最大経過時間
        alpha (float): 出力変化に対する閾値
        loss (callable): ロス関数
        subf0 (bool): 初期モデルの出力を使用するかどうか
        max_dgrad (float): 勾配の変化の最大許容値
        max_dout (float): 出力の変化の最大許容値
    """

    # 初期モデルのコピーを作成
    f = copy.deepcopy(f0)

    # モデルの出力を計算（必要に応じて初期モデルの出力を使用）
    with torch.no_grad():
        out0 = f0(x) if subf0 else 0

    # 時間刻みとモーメンタムのパラメータを初期化
    dt = 1
    step_change_dt = 0

    # ContinuousMomentum オプティマイザを初期化
    optimizer = ContinuousMomentum(f.parameters(), dt=dt, tau=tau)

    # ログ-線形間隔のチェックポイント生成器を作成
    checkpoint_generator = loglinspace(0.01, 100)
    checkpoint = next(checkpoint_generator)

    # 経過時間を計測するためのタイマーを開始
    wall = perf_counter()
    t = 0
    converged = False

    # 最初のモデルの出力と勾配を計算
    out = f(x)
    grad = gradient(loss((out - out0) * y).mean(), f.parameters())

    # トレーニングループ
    for step in itertools.count():
        # 現在のモデルとオプティマイザの状態を保存
        state = copy.deepcopy((f.state_dict(), optimizer.state_dict(), t))

        while True:
            # 最適化ステップを実行
            make_step(f, optimizer, dt, grad)
            t += dt
            current_dt = dt

            # モデルの新しい出力と勾配を計算
            new_out = f(x)
            new_grad = gradient(loss((new_out - out0) * y).mean(), f.parameters())

            # 出力の変化量を計算
            dout = (out - new_out).mul(alpha).abs().max().item()

            # 勾配の変化量を計算
            if grad.norm() == 0 or new_grad.norm() == 0:
                dgrad = 0
            else:
                dgrad = (grad - new_grad).norm().pow(2).div(grad.norm() * new_grad.norm()).item()

            # 勾配の変化量と出力の変化量が閾値以下であれば，時間刻みを増加
            if dgrad < max_dgrad and dout < max_dout:
                if dgrad < 0.5 * max_dgrad and dout < 0.5 * max_dout:
                    dt *= 1.1
                break
            # そうでない場合は，時間刻みを減少
            dt /= 10

            # 現在の状態を出力
            print("[{} +{}] [dt={:.1e} dgrad={:.1e} dout={:.1e}]".format(step, step - step_change_dt, dt, dgrad, dout), flush=True)

            # モデルとオプティマイザの状態をリストア
            step_change_dt = step
            f.load_state_dict(state[0])
            optimizer.load_state_dict(state[1])
            t = state[2]

        # 新しい出力と勾配を保存
        out = new_out
        grad = new_grad

        save = False

        # チェックポイントに達した場合，または収束した場合に状態を保存
        if step == checkpoint:
            checkpoint = next(checkpoint_generator)
            assert checkpoint > step
            save = True

        # 出力が閾値を超え，収束していない場合に収束を宣言
        if (alpha * (out - out0) * y >= 1).all() and not converged:
            converged = True
            save = True

        if save:
            state = {
                'step': step,
                'wall': perf_counter() - wall,
                't': t,
                'dt': current_dt,
                'dgrad': dgrad,
                'dout': dout,
                'norm': sum(p.norm().pow(2) for p in f.parameters()).sqrt().item(),
                'dnorm': sum((p0 - p).norm().pow(2) for p0, p in zip(f0.parameters(), f.parameters())).sqrt().item(),
                'grad_norm': grad.norm().item(),
            }

            yield f, state, converged

        # 収束した場合はトレーニングを終了
        if converged:
            break

        # 経過時間が最大経過時間を超えた場合にトレーニングを終了
        if perf_counter() > wall + max_walltime:
            break

        # 出力に NaN が含まれている場合，トレーニングを終了
        if torch.isnan(out).any():
            break



def train_kernel(ktrtr, ytr, tau, max_walltime, alpha, loss_prim, max_dgrad=math.inf, max_dout=math.inf):
    """
    線形モデル専用のトレーニング関数

    Arguments:
        ktrtr (torch.Tensor): カーネル行列
        ytr (torch.Tensor): 出力ラベル
        tau (float): モーメンタムパラメータ
        max_walltime (float): 最大経過時間
        alpha (float): 出力変化に対する閾値
        loss_prim (callable): プライムロス関数
        max_dgrad (float): 勾配の変化の最大許容値
        max_dout (float): 出力の変化の最大許容値
    """
    # 初期出力と速度ベクトルをゼロで初期化
    otr = ktrtr.new_zeros(len(ytr))
    velo = otr.clone()

    # 時間刻みとステップ変更のタイミングを初期化
    dt = 1
    step_change_dt = 0

    # ログ-線形間隔のチェックポイント生成器を作成
    checkpoint_generator = loglinspace(0.01, 100)
    checkpoint = next(checkpoint_generator)

    # 経過時間を計測するためのタイマーを開始
    wall = perf_counter()
    t = 0
    converged = False

    # 初期のプライムロスを計算
    lprim = loss_prim(otr * ytr) * ytr
    grad = ktrtr @ lprim / len(ytr)

    # トレーニングループ
    for step in itertools.count():

        # 現在の出力，速度，時間を保存
        state = copy.deepcopy((otr, velo, t))

        while True:
            # モーメンタムパラメータに基づいて速度を更新
            if tau > 0:
                x = math.exp(-dt / tau)
                velo.mul_(x).add_(-(1 - x), grad)
            elif tau < 0:
                mu = -tau
                x = (t / (t + dt)) ** mu
                velo.mul_(x).add_(-(1 - x), grad)
            else:
                velo.copy_(-grad)

            # 出力を更新
            otr.add_(dt, velo)

            t += dt
            current_dt = dt

            # 新しいプライムロスを計算し，勾配を更新
            lprim = loss_prim(otr * ytr) * ytr
            new_grad = ktrtr @ lprim / len(ytr)

            # 出力の変化量を計算
            dout = velo.mul(dt * alpha).abs().max().item()

            # 勾配の変化量を計算
            if grad.norm() == 0 or new_grad.norm() == 0:
                dgrad = 0
            else:
                dgrad = (grad - new_grad).norm().pow(2).div(grad.norm() * new_grad.norm()).item()

            # 勾配と出力の変化量が許容範囲内であれば，時間刻みを増加
            if dgrad < max_dgrad and dout < max_dout:
                if dgrad < 0.1 * max_dgrad and dout < 0.1 * max_dout:
                    dt *= 1.1
                break

            # そうでない場合は，時間刻みを減少
            dt /= 10

            # 現在の状態を出力
            print("[{} +{}] [dt={:.1e} dgrad={:.1e} dout={:.1e}]".format(step, step - step_change_dt, dt, dgrad, dout), flush=True)
            step_change_dt = step

            # 保存した状態をリストア
            otr.copy_(state[0])
            velo.copy_(state[1])
            t = state[2]

        # 勾配を新しい値で更新
        grad = new_grad

        save = False

        # チェックポイントに達した場合，または収束した場合に状態を保存
        if step == checkpoint:
            checkpoint = next(checkpoint_generator)
            assert checkpoint > step
            save = True

        # 出力が閾値を超え，収束していない場合に収束を宣言
        if (alpha * otr * ytr >= 1).all() and not converged:
            converged = True
            save = True

        if save:
            state = {
                'step': step,
                'wall': perf_counter() - wall,
                't': t,
                'dt': current_dt,
                'dgrad': dgrad,
                'dout': dout,
                'grad_norm': grad.norm().item(),
            }

            yield otr, velo, grad, state, converged

        # 収束した場合はトレーニングを終了
        if converged:
            break

        # 経過時間が最大経過時間を超えた場合にトレーニングを終了
        if perf_counter() > wall + max_walltime:
            break

        # 出力に NaN が含まれている場合，トレーニングを終了
        if torch.isnan(otr).any():
            break

`compute_kernels` 関数は，与えられたモデル $ f $ の入力データ $ x_{\text{tr}} $ と $ x_{\text{te}} $ に基づいて，カーネル行列（Gram 行列）を計算．

- $ K_{\text{trtr}} $：トレーニングデータ間のカーネル行列
- $ K_{\text{tetr}} $：テストデータとトレーニングデータ間のカーネル行列
- $ K_{\text{tete}} $：テストデータ間のカーネル行列

これらの行列は，モデルのパラメータに関する勾配を使って計算．

<br>

### モデルの勾配計算

モデル $ f $ が入力 $ x $ に対して出力を生成し，その勾配を計算．勾配の計算は以下のように行う：

1. 入力 $ x $ に対するモデルの出力を $ f(x) $ とする．
2. この出力に対するパラメータ $ \theta $ の勾配を求める：$ \nabla_\theta f(x) $

  ここで，$ \nabla_\theta f(x) $ は $ x $ に対する勾配であり，モデルのパラメータ $ \theta $ に関する勾配ベクトル．

<br>

### Gram 行列の計算

1. $ K_{\text{trtr}} $

  トレーニングデータ $ x_{\text{tr}} $ に対するカーネル行列 $ K_{\text{trtr}} $ は，各トレーニングデータポイント $ x_i $ と $ x_j $ の勾配ベクトルに基づいて計算：

  $ K_{\text{trtr}} = J_{\text{tr}} J_{\text{tr}}^T $

  ここで，$ J_{\text{tr}} $ はトレーニングデータ $ x_{\text{tr}} $ に対する勾配ベクトルを列に持つ行列．$ J_{\text{tr}} $ の $ i $-th 行は，入力 $ x_i $ に対する勾配ベクトル．

<br>

2. $ K_{\text{tetr}} $

  テストデータ $ x_{\text{te}} $ とトレーニングデータ $ x_{\text{tr}} $ とのカーネル行列 $ K_{\text{tetr}} $ は次のように計算：

  $ K_{\text{tetr}} = J_{\text{te}} J_{\text{tr}}^T $

  ここで，$ J_{\text{te}} $ はテストデータ $ x_{\text{te}} $ に対する勾配ベクトルを列に持つ行列．

<br>

3. $ K_{\text{tete}} $

  テストデータ $ x_{\text{te}} $ に対するカーネル行列 $ K_{\text{tete}} $ は、テストデータポイント $ x_i $ と $ x_j $ の勾配ベクトルに基づいて計算：

  $ K_{\text{tete}} = J_{\text{te}} J_{\text{te}}^T $

  ここで，$ J_{\text{te}} $ の $ i $-th 行は，入力 $ x_i $ に対する勾配ベクトル．

In [ ]:
# pylint: disable=no-member, C, not-callable
"""
Computes the Gram matrix of a given model
"""

def compute_kernels(f, xtr, xte):
    # from hessian import gradient

    # 新しいゼロ行列を作成
    # ktrtr: トレーニングデータ間のカーネル行列
    # ktetr: テストデータとトレーニングデータ間のカーネル行列
    # ktete: テストデータ間のカーネル行列
    ktrtr = xtr.new_zeros(len(xtr), len(xtr))
    ktetr = xtr.new_zeros(len(xte), len(xtr))
    ktete = xtr.new_zeros(len(xte), len(xte))

    params = []
    current = []

    # モデルのパラメータをサイズで降順にソートし，メモリ制限に基づいて分割
    for p in sorted(f.parameters(), key=lambda p: p.numel(), reverse=True):
        current.append(p)
        # メモリ制限に基づき，パラメータを分割
        if sum(p.numel() for p in current) > 2e9 // (8 * (len(xtr) + len(xte))):
            if len(current) > 1:
                params.append(current[:-1])
                current = current[-1:]
            else:
                params.append(current)
                current = []
    if len(current) > 0:
        params.append(current)

    # 各パラメータグループについてカーネル行列を計算
    for i, p in enumerate(params):
        print("[{}/{}] [len={} numel={}]".format(i, len(params), len(p), sum(x.numel() for x in p)), flush=True)

        # 勾配行列を初期化
        jtr = xtr.new_empty(len(xtr), sum(u.numel() for u in p))  # (P, N~) # (トレーニングデータ数, パラメータ数の合計)
        jte = xte.new_empty(len(xte), sum(u.numel() for u in p))  # (P, N~) # (テストデータ数, パラメータ数の合計)

        # トレーニングデータに対する勾配行列を計算
        for j, x in enumerate(xtr):
            jtr[j] = gradient(f(x[None]), p)  # (N~) # (パラメータ数の合計)

        # テストデータに対する勾配行列を計算
        for j, x in enumerate(xte):
            jte[j] = gradient(f(x[None]), p)  # (N~) # (パラメータ数の合計)

        # カーネル行列を更新
        ktrtr.add_(jtr @ jtr.t())  # トレーニングデータ間のカーネル行列
        ktetr.add_(jte @ jtr.t())  # テストデータとトレーニングデータ間のカーネル行列
        ktete.add_(jte @ jte.t())  # テストデータ間のカーネル行列
        del jtr, jte  # 不要になった勾配行列を削除

    return ktrtr, ktetr, ktete